## Flatness v.s. Generalization - part 2

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from keras.datasets import mnist
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K

/home/sitibanc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Data Preprocessing

In [2]:
# Load Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalize
x_train = x_train / 255
x_test = x_test / 255
# Get One-Hot Labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

### Build Model

In [3]:
def build_model(print_summary=False):
    '''
    Build model
    Args:
        print_summary: bool, whether or not to print model summary, default: False
    Returns:
        model: keras model
    '''
    
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28), name="input"))
    model.add(Dense(16, activation="relu", name="fc1"))
    model.add(Dense(16, activation="relu", name="fc2"))
    model.add(Dense(10, activation="softmax", name="output"))

    if print_summary:
        model.summary()
    
    return model

### Train Model

#### Batch size=8

In [4]:
model8 = build_model(True)
model8.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history8 = model8.fit(x_train[:3000], y_train[:3000], batch_size=8, epochs=100, validation_data=(x_test[:1000], y_test[:1000]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Flatten)              (None, 784)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 16)                12560     
_________________________________________________________________
fc2 (Dense)                  (None, 16)                272       
_________________________________________________________________
output (Dense)               (None, 10)                170       
Total params: 13,002
Trainable params: 13,002
Non-trainable params: 0
_________________________________________________________________
Train on 3000 samples, validate on 1000 samples
Epoch 1/100
3000/3000 [==============================] - 0s 145us/step - loss: 1.2981 - acc: 0.6137 - val_loss: 0.8213 - val_acc: 0.7670
Epoch 2/100
3000/3000 [==============================] - 0s 102us/step - loss: 0.5264

Epoch 54/100
3000/3000 [==============================] - 0s 103us/step - loss: 8.4542e-04 - acc: 1.0000 - val_loss: 0.6845 - val_acc: 0.8850
Epoch 55/100
3000/3000 [==============================] - 0s 103us/step - loss: 7.1460e-04 - acc: 1.0000 - val_loss: 0.6899 - val_acc: 0.8890
Epoch 56/100
3000/3000 [==============================] - 0s 103us/step - loss: 6.2542e-04 - acc: 1.0000 - val_loss: 0.6913 - val_acc: 0.8910
Epoch 57/100
3000/3000 [==============================] - 0s 103us/step - loss: 5.6095e-04 - acc: 1.0000 - val_loss: 0.6949 - val_acc: 0.8880
Epoch 58/100
3000/3000 [==============================] - 0s 103us/step - loss: 5.0451e-04 - acc: 1.0000 - val_loss: 0.7019 - val_acc: 0.8890
Epoch 59/100
3000/3000 [==============================] - 0s 103us/step - loss: 4.5662e-04 - acc: 1.0000 - val_loss: 0.7013 - val_acc: 0.8880
Epoch 60/100
3000/3000 [==============================] - 0s 104us/step - loss: 4.0652e-04 - acc: 1.0000 - val_loss: 0.7055 - val_acc: 0.8860
Epoch 

#### Batch size=64

In [ ]:
model64 = build_model(True)
model64.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history64 = model64.fit(x_train[:3000], y_train[:3000], batch_size=8, epochs=100, validation_data=(x_test[:1000], y_test[:1000]))

#### Batch size=128

In [ ]:
model128 = build_model(True)
model128.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history128 = model128.fit(x_train[:3000], y_train[:3000], batch_size=8, epochs=100, validation_data=(x_test[:1000], y_test[:1000]))

#### Batch size=512

In [ ]:
model512 = build_model(True)
model512.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history512 = model512.fit(x_train[:3000], y_train[:3000], batch_size=8, epochs=100, validation_data=(x_test[:1000], y_test[:1000]))

#### Batch size=1024

In [ ]:
model1024 = build_model(True)
model1024.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history1024 = model1024.fit(x_train[:3000], y_train[:3000], batch_size=8, epochs=100, validation_data=(x_test[:1000], y_test[:1000]))

### Calculate Sensitivity

In [ ]:
def cal_sensitivity(model, x, y):
    # Define tensorflow placeholder
    input_tensors = [
        model.inputs[0], # input
        model.sample_weights[0], # sample weights
        model.targets[0], # labels
        K.learning_phase() # train or test mode
    ]
    sensitivity = K.gradients(model.total_loss, model.inputs[0])
    # Define K.function()
    get_gradients = K.function(inputs=input_tensors, outputs=sensitivity)

    inputs = [
        x, # X input data
        np.ones((x.shape[0],)), # sample weights
        y, # y labels
        0 # learning phase in TEST mode
    ]
    # Call K.function()
    g = get_gradients(inputs)
    # Apply 2-norm
    g_0 = np.sum(g[0]**2)
    g_all = np.sqrt(g_0)
    return g_all

In [ ]:
sensitivity = []
sensitivity.append(cal_sensitivity(model8, x_train[0].reshape(1,28,28), y_train[0].reshape(1,10)))